This is the code that will be imported into our Raspberry Pi to process and send images to our computer for classification

In [2]:
# Import packages

# reload our external packages (tracker)
import importlib

import numpy as np
import cv2
import os
from skimage import data, filters
import tracker as track

importlib.reload(track)

# Directory to save training images

# base_dir = "/Users/williamlee/Documents/Git Repos/Lego-Brick-Sorter/Imaging Pipeline/testImages"
base_dir = "/home/billiam/Documents/Lego_Sorter/TestPipelineImages"
os.chdir(base_dir)
print(os.listdir())

[]


**MOG2 APPROACH**

In [38]:
cap = cv2.VideoCapture("/home/billiam/Documents/Repos/Lego-Brick-Sorter/Imaging Pipeline/white.avi")
frame_counter = 0

# Create BG Subtraction Function with set parameters
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)
fgbg.setVarThreshold(30) #The threshold of what is detected as not background (150 can detect regular pieces, but not white)

while cap.isOpened():
    ret, frame = cap.read()
    
    # Loop video code
    frame_counter += 1
    if frame_counter == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        frame_counter = 0 #Or whatever as long as it is the same as next line
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    # Exit if frame not correctly
    if not ret:
        print("Frame not read correctly. Exiting...")
        break
    
    #0.Pre-Process Image
    saturation_factor = 3 # Factor to saturate image by
    value_factor = 1.2     # Factor to increase value of image by

    # Crop and adjust saturation of image. Blur to reduce noise from saturation
    #frame = frame[y_top_crop:y_bottom_crop, x_left_crop:x_right_crop]
    adjusted = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV).astype("float32")
    (h, s, v) = cv2.split(adjusted)
    s = s * saturation_factor
    s = np.clip(s,0,255)
    v = v * value_factor
    v = np.clip(v,0,255)
    adjusted = cv2.merge([h,s,v])
    frame = cv2.cvtColor(adjusted.astype("uint8"), cv2.COLOR_HSV2BGR)
    frame = cv2.GaussianBlur(frame, (5, 5), cv2.BORDER_DEFAULT)    
        
    # Apply MOG2 to Frame
    fgmask = fgbg.apply(frame)

    # Find contours on MOG2 mask and draw on frame copy
    frame_copy = frame.copy()
    contours, hierarchy = cv2.findContours(image=fgmask, mode=cv2.RETR_CCOMP, method=cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(frame_copy, contours, -1, (0,255,0), 3)
    
    window_factor = 0.75
    frame = cv2.resize(frame, None, fx=window_factor, fy=window_factor, interpolation=cv2.INTER_AREA)
    fgmask = cv2.resize(fgmask, None, fx=window_factor, fy=window_factor, interpolation=cv2.INTER_AREA)
    cv2.imshow('frame', frame)
    cv2.imshow('MOG2', fgmask)
    cv2.imshow('Contours', frame_copy)
    if cv2.waitKey(50) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

<br></br>
**DIFFERENCE OF GAUSSIANS APPROACH**

Notes:
Kind of works! I think I need to work with the thresholding a little bit. So far I've only adjusted the extent threshold.
Have to mess with the k and s values for the blurs to maybe get more accurate. Works well with regular colored pieces.

In [86]:
def dog(img, k1, s1, k2, s2):
    b1 = cv2.GaussianBlur(img,(k1, k1), s1)
    b2 = cv2.GaussianBlur(img,(k2, k2), s2)
    return b1 - b2

cap = cv2.VideoCapture("/home/billiam/Documents/Repos/Lego-Brick-Sorter/Imaging Pipeline/white.avi")
frame_counter = 0

while cap.isOpened():
    ret, frame = cap.read()
    
    # Loop video code
    frame_counter += 1
    if frame_counter == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        frame_counter = 0 #Or whatever as long as it is the same as next line
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    # Color processing:
    saturation_factor = 3 # Factor to saturate image by
    value_factor = 1      # Factor to increase value of image by
    adjusted = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV).astype("float32")
    (h, s, v) = cv2.split(adjusted)
    s = s * saturation_factor
    s = np.clip(s,0,255)
    v = v * value_factor
    v = np.clip(v,0,255)
    adjusted = cv2.merge([h,s,v])
    frame = cv2.cvtColor(adjusted.astype("uint8"), cv2.COLOR_HSV2BGR)
    
    # Exit if frame not correctly
    if not ret:
        print("Frame not read correctly. Exiting...")
        break
    
    grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    dog_img = dog(grey, 7, 7, 17, 13)
    th = cv2.threshold(dog_img ,127,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    contours, hierarchy = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    frame_copy = frame.copy()
    
    #cv2.drawContours(frame_copy, contours, -1, (0,255,0), 3)
    
    for c in contours:
        area = cv2.contourArea(c)
        if area > 300:
            x,y,w,h = cv2.boundingRect(c)
            extent = int(area)/(w*h)
            #if extent > 0.1:
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)
            box = np.intp(box)
            cv2.drawContours(frame_copy,[box], 0, (0,255,0), 4)
            
    
    cv2.imshow("dog frames", frame_copy)
    cv2.imshow('grayscale', grey)
    cv2.imshow("dog edges", dog_img)
    
    if cv2.waitKey(100) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

**Sobel and Canny Edge Detection**

In [97]:
cap = cv2.VideoCapture("/home/billiam/Documents/Repos/Lego-Brick-Sorter/Imaging Pipeline/white.avi")
frame_counter = 0

while cap.isOpened():
    ret, frame = cap.read()
    
    # Loop video code
    frame_counter += 1
    if frame_counter == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        frame_counter = 0 #Or whatever as long as it is the same as next line
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        
    # Color processing:
    saturation_factor = 3 # Factor to saturate image by
    value_factor = 1      # Factor to increase value of image by
    adjusted = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV).astype("float32")
    (h, s, v) = cv2.split(adjusted)
    s = s * saturation_factor
    s = np.clip(s,0,255)
    v = v * value_factor
    v = np.clip(v,0,255)
    adjusted = cv2.merge([h,s,v])
    frame = cv2.cvtColor(adjusted.astype("uint8"), cv2.COLOR_HSV2BGR)
    
    # Exit if frame not correctly
    if not ret:
        print("Frame not read correctly. Exiting...")
        break
        
    # Grayscale convert
    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Blur img
    img_blur = cv2.GaussianBlur(img_gray, (3,3), 0)
    
    # # Sobel Edge Detection
    # sobelx = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
    # sobely = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
    # sobelxy = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection
    # # Display Sobel Edge Detection Images
    # cv2.imshow('Sobel X', sobelx)
    # cv2.imshow('Sobel Y', sobely)
    # cv2.imshow('Sobel X Y using Sobel() function', sobelxy)
    
    # Canny Edge Detection
    edges = cv2.Canny(image=img_blur, threshold1=0, threshold2=35) # Canny Edge Detection
    
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    frame_copy = frame.copy()
    #cv2.drawContours(frame_copy, contours, -1, (0,255,0), 3)
    
    #cv2.drawContours(frame_copy, contours, -1, (0,255,0), 3)
    
    for c in contours:
        area = cv2.contourArea(c)
        if area > 300:
            x,y,w,h = cv2.boundingRect(c)
            extent = int(area)/(w*h)
            #if extent > 0.1:
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)
            box = np.intp(box)
            cv2.drawContours(frame_copy,[box], 0, (0,255,0), 4)
    
    # Display Canny Edge Detection Image
    cv2.imshow('Canny Edge Detection', edges)
    cv2.imshow('Contours', frame_copy)
    if cv2.waitKey(100) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [33]:
# pieceNum = "3021"
# os.chdir(pieceNum)
# Open Video
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(cv2.CAP_V4L2)

# Adjust Camera Settings
cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 1) # Manual exposure mode
cap.set(cv2.CAP_PROP_EXPOSURE, 20)
cap.set(cv2.CAP_PROP_AUTO_WB, 0) # Disable automatic white balance (Not sure if this is working)

# Create Tracker Object
tracker = track.EuclideanDistTracker()

# Create BG Subtraction Function with set parameters
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)
fgbg.setVarThreshold(30) #The threshold of what is detected as not background (150 can detect regular pieces, but not white)

# Create Window
cv2.startWindowThread()

while(True):
    # print("in loop")
    # Capture video frame by frame
    ret, frame = cap.read()
    
    # 0: Pre-Process Image
    window_factor = 1      # Size factor of display window
    x_left_crop = 0        # Left side window crop
    x_right_crop = 1920     # Right side window crop
    y_top_crop = 0          # Top side window crop
    y_bottom_crop = 1080    # Bottom side window crop
    saturation_factor = 6 # Factor to saturate image by
    value_factor = 1.2      # Factor to increase value of image by

    # Crop and adjust saturation of image. Blur to reduce noise from saturation
    #frame = frame[y_top_crop:y_bottom_crop, x_left_crop:x_right_crop]
    adjusted = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV).astype("float32")
    (h, s, v) = cv2.split(adjusted)
    s = s * saturation_factor
    s = np.clip(s,0,255)
    v = v * value_factor
    v = np.clip(v,0,255)
    adjusted = cv2.merge([h,s,v])
    frame = cv2.cvtColor(adjusted.astype("uint8"), cv2.COLOR_HSV2BGR)
    frame = cv2.GaussianBlur(frame, (5, 5), 0)

    # Apply MOG2 to Frame
    fgmask = fgbg.apply(frame)

    # Adjust size
    #frame = cv2.resize(frame, None, fx=window_factor, fy=window_factor, interpolation=cv2.INTER_AREA)
    #fgmask = cv2.resize(fgmask, None, fx=window_factor, fy=window_factor, interpolation=cv2.INTER_AREA)

    
    # 1: Object Detection
    min_area = 300 # Min area of contour to detect as piece
    
    # Iterate through discovered contours and add detections
    contours, hierarchy = cv2.findContours(image=fgmask, mode=cv2.RETR_CCOMP, method=cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    image_copy = frame.copy()
    contourFrame = frame.copy()
    
    cv2.drawContours(contourFrame, contours, -1, (0,255,0), 3)
    
    for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            # If contour overlap with border then skip
            if((x == 0) or (y == 0) or (x+w >= frame.shape[1]) or (y+h >= frame.shape[0])):
                continue
            
            area = cv2.contourArea(contour)
            if(area > min_area):
                detections.append([x, y, w, h])
                
    # In order to detect white pieces, we need to write a function that takes all the contours, looks at the ones that make the size criteria, and then
    # finds all nearby contours and adds them to the contour space. The white pieces being detected as a bunch of small contours
    
    # 2: Object Tracking
    min_age = 1 # Min age of object to classify as brick
    interval = 1 # Frequency of images to save after being classified as brick
    
    box_ids = tracker.update(detections)
    for box_id in box_ids:
        x, y, w, h, id, age, uniqueId = box_id
        # If a box is "old enough" box is determined to be a piece --> take pictures
        if age > min_age:
            larger = w
            if(h > w):
                larger = h
            cv2.rectangle(image_copy, (x,y), (x+larger, y+larger), (0, 255, 0), 3)
            cv2.putText(image_copy, "PIECE IS HERE" + " time:" + str(age) + " id:" + str(uniqueId),(x,y-10),0,0.5,(255, 0, 0), 2)
            
            # Save bounding box as image every X frames
            roi = frame[y:y+larger, x:x+larger]
            true_age = age - min_age
            
            if(true_age % interval == 0):
                print(true_age)
                print("save image")
                cv2.imwrite(str(uniqueId) + "_" + str(age) + ".png", roi)
            
    
    cv2.imshow('Feed with bounding boxes', image_copy)
    cv2.imshow('MOG2 Output', fgmask)
    cv2.imshow('Contours', contourFrame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()

1
save image
2
save image
3
save image
4
save image
5
save image
6
save image
7
save image
1
save image
2
save image
3
save image
4
save image
1
save image
2
save image
3
save image
1
save image
